Datensatz öffnen

In [1]:
import pandas as pd
df = pd.read_csv('movies.csv')
print(df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Jahr als separates Feature

In [2]:
# Jahr extrahieren in neue Spalte 'year'
df['year'] = df['title'].str.extract(r'\((\d{4})\)')

# Klammerjahr aus dem Titel entfernen
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()

# Ergebnis anzeigen
print(df.head(10))


   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   
5        6                         Heat   
6        7                      Sabrina   
7        8                 Tom and Huck   
8        9                 Sudden Death   
9       10                    GoldenEye   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  
5                        Action|Crime|Thriller  1995  
6                               Comedy|Romance  1995  
7                           Adventure|Children  1995  
8                              

# Prototyp

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Daten vorbereiten
df = pd.read_csv('movies.csv')
df['year'] = df['title'].str.extract(r'\((\d{4})\)')
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df['year'] = df['year'].astype('Int64')

# Genre-Vektoren (einmalig!)
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(df['genres'])

# Mapping von Titel zu Index
title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# User Input (3 Titel)
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

# Finde die Vektoren der eingegebenen Filme
valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
else:
    # Durchschnittlicher Genre-Vektor (mit Umwandlung zu dichten Array)
    user_vector = genre_matrix[valid_indices].mean(axis=0).A.flatten()

    # Ähnlichkeit zwischen User-Vektor und allen Filmen berechnen
    similarities = cosine_similarity(user_vector.reshape(1, -1), genre_matrix).flatten()

    # Top-N Empfehlungen (ausgenommen Eingaben)
    top_n = 5
    recommendations = similarities.argsort()[::-1]
    recommendations = [i for i in recommendations if i not in valid_indices][:top_n]

    print("\n🎬 Empfehlungen:")
    print(df.iloc[recommendations][['title', 'genres', 'year']])


c:\Users\deicc\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bitte gib 3 Filmtitel ein:
⚠️  Film nicht gefunden: jökfdjlsökdajfdskjf

🎬 Empfehlungen:
                                       title  \
61889                              UglyDolls   
30639    Scooby-Doo! Mask of the Blue Falcon   
23228                      The Magic Crystal   
72526              Legends of Valhalla: Thor   
49160  Puss in Book: Trapped in an Epic Tale   

                                            genres  year  
61889  Adventure|Animation|Children|Comedy|Fantasy  2019  
30639  Adventure|Animation|Children|Comedy|Fantasy  2012  
23228  Adventure|Animation|Children|Comedy|Fantasy  2011  
72526  Adventure|Animation|Children|Comedy|Fantasy  2011  
49160  Adventure|Animation|Children|Comedy|Fantasy  2017  


Timemachine (separiert auch zwischen alte und neue Movies)

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Daten vorbereiten
df = pd.read_csv('movies.csv')
df['year'] = df['title'].str.extract(r'\((\d{4})\)')
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df['year'] = df['year'].astype('Int64')

# Zeilen ohne Jahr ausschließen
df = df.dropna(subset=['year'])

# Genre-Vektoren (einmalig!)
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(df['genres'])

# Mapping von Titel zu Index
title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# User Input (3 Titel)
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

# Finde die Vektoren der eingegebenen Filme
valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
else:
    # Durchschnittlicher Genre-Vektor (mit Umwandlung zu dichten Array)
    user_vector = genre_matrix[valid_indices].mean(axis=0).A.flatten()

    # Ähnlichkeit zwischen User-Vektor und allen Filmen berechnen
    similarities = cosine_similarity(user_vector.reshape(1, -1), genre_matrix).flatten()

    # Top-N Empfehlungen (ausgenommen Eingaben)
    top_n = 5
    recommendations = similarities.argsort()[::-1]
    recommendations = [i for i in recommendations if i not in valid_indices][:top_n]

    print("\n🎬 Allgemeine Empfehlungen:")
    print(df.iloc[recommendations][['title', 'genres', 'year']])

    # Time Machine: alte und neue Filme
    old_mask = df['year'] < 2010
    new_mask = df['year'] >= 2010

    df_old = df[old_mask].reset_index(drop=True)
    df_new = df[new_mask].reset_index(drop=True)

    genre_old = genre_matrix[old_mask.values]
    genre_new = genre_matrix[new_mask.values]

    # Ähnlichkeiten berechnen
    sim_old = cosine_similarity(user_vector.reshape(1, -1), genre_old).flatten()
    sim_new = cosine_similarity(user_vector.reshape(1, -1), genre_new).flatten()

    rec_old = sim_old.argsort()[::-1][:top_n]
    rec_new = sim_new.argsort()[::-1][:top_n]

    print("\n🕰️ Empfehlungen aus der Vergangenheit (vor 2010):")
    print(df_old.iloc[rec_old][['title', 'genres', 'year']])

    print("\n🚀 Empfehlungen aus der Zukunft (2010 oder später):")
    print(df_new.iloc[rec_new][['title', 'genres', 'year']])


c:\Users\deicc\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bitte gib 3 Filmtitel ein:

🎬 Allgemeine Empfehlungen:
                                                  title  \
9951   DuckTales: The Movie - Treasure of the Lost Lamp   
56661                                   Penguin Highway   
80145                 Hotel Transylvania: Transformania   
83783                                My Father's Dragon   
51298                           Olaf's Frozen Adventure   

                                            genres  year  
9951   Adventure|Animation|Children|Comedy|Fantasy  1990  
56661  Adventure|Animation|Children|Comedy|Fantasy  2018  
80145  Adventure|Animation|Children|Comedy|Fantasy  2022  
83783  Adventure|Animation|Children|Comedy|Fantasy  2022  
51298  Adventure|Animation|Children|Comedy|Fantasy  2017  

🕰️ Empfehlungen aus der Vergangenheit (vor 2010):
                         title                                       genres  \
0                    Toy Story  Adventure|Animation|Children|Comedy|Fantasy   
10775                Wild, 

# Code mit Tags
Im folgenden Code werden Tags inkludiuert, da ist das folgende klar geworden.

Manche Movies haben KEINE Tags und manche haben SEHR VIELE, das verursacht PROBLEMS. Wenn Users z.B. Inputs verwenden, welche keine Tags haben, dann werden vom Code Empfehlungen preferiert, die AUCH keine Tags haben. Das scheint nicht sinnvoll zu sein.

Das Problem entsteht dadurch, dass Features "Genre" und "Tags" kombiniert werden, und dann das empfohlen wird, was am meisten Gemeinsamkeiten zwischen den Films hat. Sinnvoller wäre, wenns eine gewisse Hierarchie geben würde. Beispielsweise, dass die Genres Wichtiger sind als Tags. Allerdings scheint es sinnvoller die Tags komplett zu ignorieren.

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# 1. Daten laden und vorbereiten
# -----------------------------
df_movies = pd.read_csv('movies.csv')
df_tags = pd.read_csv('tags.csv')

# Jahr extrahieren und Titel bereinigen
df_movies['year'] = df_movies['title'].str.extract(r'\((\d{4})\)')
df_movies['title'] = df_movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df_movies['year'] = pd.to_numeric(df_movies['year'], errors='coerce')

# Relevante Spalten aus Tags behalten und gruppieren
df_tags = df_tags[['movieId', 'tag']]
df_tags = df_tags.dropna(subset=['tag'])  # leere Tags entfernen
tags_grouped = df_tags.groupby('movieId')['tag'].apply(lambda tags: '|'.join(tags.unique())).reset_index()
tags_grouped.columns = ['movieId', 'tags']

# Tags mit Movie-Daten kombinieren
df = df_movies.merge(tags_grouped, on='movieId', how='left')
df['tags'] = df['tags'].fillna('')  # Leere Tags auffüllen

# -----------------------------
# 2. Feature-Vektorisierung
# -----------------------------
# Genres + Tags zu einem kombinierten Feature
df['combined'] = df['genres'].fillna('') + '|' + df['tags']

vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
combined_matrix = vectorizer.fit_transform(df['combined'])

# Mapping Titel -> Index
title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# -----------------------------
# 3. Benutzereingabe
# -----------------------------
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
else:
    user_vector = combined_matrix[valid_indices].mean(axis=0).A.flatten()

    # Jahr-Masken (nur Filme mit gültigem Jahr)
    valid_year_mask = df["year"].notna()
    old_mask = valid_year_mask & (df["year"] < 2010)
    new_mask = valid_year_mask & (df["year"] >= 2010)

    df_old = df[old_mask].reset_index(drop=True)
    df_new = df[new_mask].reset_index(drop=True)

    matrix_old = combined_matrix[old_mask.values]
    matrix_new = combined_matrix[new_mask.values]

    # Ähnlichkeiten berechnen
    sim_old = cosine_similarity(user_vector.reshape(1, -1), matrix_old).flatten()
    sim_new = cosine_similarity(user_vector.reshape(1, -1), matrix_new).flatten()

    # Top-Empfehlungen (alte & neue)
    top_n = 5
    rec_old = sim_old.argsort()[::-1][:top_n]
    rec_new = sim_new.argsort()[::-1][:top_n]

    df['tags'] = df['tags'].fillna('no tags')

    print("\n🎬 Empfehlungen (vor 2010):")
    print(df_old.iloc[rec_old][['title', 'genres', 'tags', 'year']])

    print("\n🎬 Empfehlungen (ab 2010):")
    print(df_new.iloc[rec_new][['title', 'genres', 'tags', 'year']])


c:\Users\deicc\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bitte gib 3 Filmtitel ein:
⚠️  Film nicht gefunden: f
⚠️  Film nicht gefunden: f
⚠️  Film nicht gefunden: f
❌ Keine gültigen Filme eingegeben. Abbruch.


# Ratings

In [4]:
import pandas as pd
import os

# Verzeichnis, in dem die CSV-Dateien liegen
ratings_dir = 'ratings'

# Liste der Dateipfade (ratings/ratings_0.csv bis ratings/ratings_32.csv)
file_paths = [os.path.join(ratings_dir, f'ratings_{i}.csv') for i in range(33)]

# CSVs einlesen und nur movieId & rating behalten
all_dfs = []
for path in file_paths:
    df = pd.read_csv(path, usecols=['movieId', 'rating'])
    all_dfs.append(df)

# Kombinieren aller DataFrames
all_ratings = pd.concat(all_dfs, ignore_index=True)

# Durchschnittliches Rating pro movieId berechnen
average_ratings = all_ratings.groupby('movieId', as_index=False)['rating'].mean()
average_ratings.rename(columns={'rating': 'mean_rating'}, inplace=True)

# Vorschau anzeigen
print(average_ratings.head())

# Optional: Speichern
# average_ratings.to_csv('average_movie_ratings.csv', index=False)


   movieId  mean_rating
0        1     3.897438
1        2     3.275758
2        3     3.139447
3        4     2.845331
4        5     3.059602


In [10]:
# Sortieren nach Rating (absteigend)
average_ratings_sorted = average_ratings.sort_values(by='mean_rating', ascending=False)

# Top 5 anzeigen
print("🎬 Top 5 Filme mit höchstem durchschnittlichem Rating:")
print(average_ratings_sorted.head(10000))

🎬 Top 5 Filme mit höchstem durchschnittlichem Rating:
       movieId  mean_rating
57193   199103          5.0
77007   268476          5.0
44589   171849          5.0
76896   268054          5.0
76900   268090          5.0
...        ...          ...
37299   154650          3.8
62370   211251          3.8
37082   154200          3.8
62074   210537          3.8
38659   158270          3.8

[10000 rows x 2 columns]


# Code mit Ratings

In [12]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# 1. Filme laden
# -----------------------------
df = pd.read_csv('movies.csv')
df['year'] = df['title'].str.extract(r'\((\d{4})\)')
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df['year'] = df['year'].astype('Int64')
df = df.dropna(subset=['year'])

# -----------------------------
# 2. Ratings aggregieren
# -----------------------------
ratings_dir = 'ratings'
rating_files = [os.path.join(ratings_dir, f'ratings_{i}.csv') for i in range(33)]

all_dfs = []
for path in rating_files:
    df_r = pd.read_csv(path, usecols=['movieId', 'rating'])
    all_dfs.append(df_r)

all_ratings = pd.concat(all_dfs, ignore_index=True)
mean_ratings = all_ratings.groupby('movieId', as_index=False)['rating'].mean()
mean_ratings.rename(columns={'rating': 'mean_rating'}, inplace=True)

# -----------------------------
# 3. Daten kombinieren
# -----------------------------
df = df.merge(mean_ratings, on='movieId', how='left')

# -----------------------------
# 4. Genre-Vektoren
# -----------------------------
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(df['genres'])

title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# -----------------------------
# 5. Benutzer-Input
# -----------------------------
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
else:
    try:
        min_rating = float(input("Mindest-Rating (z.B. 2.0): "))
        max_rating = float(input("Maximal-Rating (z.B. 4.5): "))
    except ValueError:
        print("⚠️ Ungültiger Zahlenwert. Abbruch.")
        exit()

    # -----------------------------
    # 6. Empfehlungslogik
    # -----------------------------
    user_vector = genre_matrix[valid_indices].mean(axis=0).A.flatten()
    similarities = cosine_similarity(user_vector.reshape(1, -1), genre_matrix).flatten()

    top_n = 5
    recommendations = similarities.argsort()[::-1]
    recommendations = [
        i for i in recommendations
        if i not in valid_indices and
        pd.notna(df.iloc[i]['mean_rating']) and
        min_rating <= df.iloc[i]['mean_rating'] <= max_rating
    ][:top_n]

    if not recommendations:
        print("\n❌ Keine passenden Empfehlungen im gewünschten Rating-Bereich.")
    else:
        print("\n🎬 Allgemeine Empfehlungen:")
        print(df.iloc[recommendations][['title', 'genres', 'year', 'mean_rating']])

        # Time Machine
        old_mask = (df['year'] < 2010) & (df['mean_rating'] >= min_rating) & (df['mean_rating'] <= max_rating)
        new_mask = (df['year'] >= 2010) & (df['mean_rating'] >= min_rating) & (df['mean_rating'] <= max_rating)

        df_old = df[old_mask].reset_index(drop=True)
        df_new = df[new_mask].reset_index(drop=True)

        genre_old = vectorizer.transform(df_old['genres'])
        genre_new = vectorizer.transform(df_new['genres'])

        sim_old = cosine_similarity(user_vector.reshape(1, -1), genre_old).flatten()
        sim_new = cosine_similarity(user_vector.reshape(1, -1), genre_new).flatten()

        rec_old = sim_old.argsort()[::-1][:top_n]
        rec_new = sim_new.argsort()[::-1][:top_n]

        print("\n🕰️ Empfehlungen aus der Vergangenheit (vor 2010):")
        print(df_old.iloc[rec_old][['title', 'genres', 'year', 'mean_rating']])

        print("\n📽️ Neuere Filmempfehlungen (ab 2010)")
        print(df_new.iloc[rec_new][['title', 'genres', 'year', 'mean_rating']])


c:\Users\deicc\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bitte gib 3 Filmtitel ein:

🎬 Allgemeine Empfehlungen:
                                    title  \
59866                        Missing Link   
22646          Toy Story Toons: Small Fry   
22645  Toy Story Toons: Hawaiian Vacation   
28329                      Brother Bear 2   
30471                   The Good Dinosaur   

                                            genres  year  mean_rating  
59866  Adventure|Animation|Children|Comedy|Fantasy  2019     3.011952  
22646  Adventure|Animation|Children|Comedy|Fantasy  2011     3.077236  
22645  Adventure|Animation|Children|Comedy|Fantasy  2011     3.043165  
28329  Adventure|Animation|Children|Comedy|Fantasy  2006     3.101240  
30471  Adventure|Animation|Children|Comedy|Fantasy  2015     3.085205  

🕰️ Empfehlungen aus der Vergangenheit (vor 2010):
                                                  title  \
3774                                     Brother Bear 2   
2535   Asterix and the Vikings (Astérix et les Vikings)   
1406  Asterix 

# Zeitaufwand Optimierung

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Ordnerpfade vorbereiten (relativ zum Skript) ---
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
RATINGS_DIR = os.path.join(BASE_DIR, 'ratings')
MOVIES_PATH = os.path.join(BASE_DIR, 'movies.csv')

# --- Filme laden und bereinigen ---
df = pd.read_csv(MOVIES_PATH)
df['year'] = df['title'].str.extract(r'\((\d{4})\)')
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df['year'] = df['year'].astype('Int64')
df = df.dropna(subset=['year'])

# --- Genres vektorisieren ---
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(df['genres'])
genre_matrix_dense = genre_matrix.toarray()

# --- Mapping: Titel zu Index ---
title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# --- Ratings einlesen & mittlere Bewertung berechnen ---
ratings_list = []
for i in range(33):
    path = os.path.join(RATINGS_DIR, f"ratings_{i}.csv")
    df_r = pd.read_csv(path, usecols=['movieId', 'rating'])
    ratings_list.append(df_r)

ratings_all = pd.concat(ratings_list, ignore_index=True)
mean_ratings = ratings_all.groupby('movieId')['rating'].mean().reset_index()
mean_ratings.rename(columns={'rating': 'mean_rating'}, inplace=True)

# --- Ratings mit Filmen verbinden ---
df = pd.merge(df, mean_ratings, on='movieId', how='left')

# --- Nutzereingabe ---
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

try:
    min_rating = float(input("Minimale Bewertung (z. B. 2.0): ").strip())
    max_rating = float(input("Maximale Bewertung (z. B. 4.5): ").strip())
except ValueError:
    print("❌ Ungültiger Bewertungsbereich. Abbruch.")
    exit()

# --- Gültige Filme überprüfen ---
valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
    exit()

# --- Benutzervektor berechnen ---
user_vector = np.asarray(genre_matrix_dense[valid_indices].mean(axis=0))

# --- Hauptempfehlung (Top-N mit Ratingfilter, Eingaben ausschließen) ---
similarities = cosine_similarity(user_vector.reshape(1, -1), genre_matrix_dense).flatten()
mask_main = (
    ~np.isin(np.arange(len(df)), valid_indices) &
    df['mean_rating'].between(min_rating, max_rating)
)
top_main = np.argsort(similarities[mask_main])[::-1][:5]
recommendations_main = df[mask_main].iloc[top_main]

print("\n🎬 Allgemeine Empfehlungen:")
print(recommendations_main[['title', 'genres', 'year', 'mean_rating']])

# --- Alte und neue Filme trennen ---
df_old = df[df['year'] < 2010].reset_index(drop=True)
df_new = df[df['year'] >= 2010].reset_index(drop=True)

genre_old = genre_matrix_dense[df['year'] < 2010]
genre_new = genre_matrix_dense[df['year'] >= 2010]

# --- Klassische Empfehlungen ---
sim_old = cosine_similarity(user_vector.reshape(1, -1), genre_old).flatten()
mask_old = df_old['mean_rating'].between(min_rating, max_rating)
top_old = np.argsort(sim_old[mask_old])[::-1][:5]
recommendations_old = df_old[mask_old].iloc[top_old]

print("\n📼 Klassiker (vor 2010):")
print(recommendations_old[['title', 'genres', 'year', 'mean_rating']])

# --- Moderne Empfehlungen ---
sim_new = cosine_similarity(user_vector.reshape(1, -1), genre_new).flatten()
mask_new = df_new['mean_rating'].between(min_rating, max_rating)
top_new = np.argsort(sim_new[mask_new])[::-1][:5]
recommendations_new = df_new[mask_new].iloc[top_new]

print("\n📱 Moderne Empfehlungen (ab 2010):")
print(recommendations_new[['title', 'genres', 'year', 'mean_rating']])


NameError: name '__file__' is not defined